# Create point defects in a bulk material should be the name of the notebook

Create a vacancy, add substitution or interstitial atom to provided Material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set defects parameters in cell 2.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create the Defect: Add a vacancy defect to the bulk structure
2. Visualize the Defect: Visualize the defect structure

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [1]:
DEFECT_TYPE = "substitution"  # (e.g. "vacancy", "substitution", "interstitial")
SITE_ID = 0  # Site index of the defect
POSITION = [0, 0.0, 0.0]  # Position of the defect in crystal coordinates
CHEMICAL_ELEMENT = "Cu"  # Element to be placed at the site (ignored for vacancy)
SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 3]] 

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [2]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_point_defect.ipynb", "../../config.yml")

### 1.3. Get input material and 
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [3]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))

Data from 0-Ni has been read successfully.
Data from 1-Graphene has been read successfully.
Data from 2-WS2 has been read successfully.
Data from 3-BN has been read successfully.
Data from 4-Te2Mo has been read successfully.
Data from 5-HfO2 has been read successfully.
Data from C2(001)-Ni4(111), Interface, Strain 0.105% has been read successfully.
Data from C2(001)-Ni4(111), Interface, Strain 0.105pc has been read successfully.
Data from C2(001)-Ni4(111), Interface, Strain 0.105pct has been read successfully.
Data from Cu1 Ni26 C18, Substitution Cu Defect has been read successfully.
Data from Cu1 Ni26, Substitution Cu Defect has been read successfully.
Data from Cu1 Ni26 has been read successfully.
Data from Cu1 Ni27 C18, Interstitial Cu Defect has been read successfully.
Data from Cu1 Ni27, Interstitial Cu Defect has been read successfully.
Data from Ni26, Vacancy  Defect has been read successfully.
Data from Ni26, Vacancy Cu Defect has been read successfully.
Data from Ni26, Vacancy

### 1.4. Create and preview Supercell

In [4]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Ni27 - Material - rotation: 0x', layout=Layout(align_self='cente…

## 2. Create the Defect
### 2.1. Set defect parameters

In [5]:
from mat3ra.made.tools.build.defect import PointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import PointDefectBuilderParameters

defect_configuration_with_site_id = PointDefectConfiguration.from_site_id(crystal=supercell, defect_type=DEFECT_TYPE, site_id=SITE_ID, chemical_element=CHEMICAL_ELEMENT)
defect_configuration_with_position = PointDefectConfiguration(crystal=supercell, defect_type="interstitial", position=POSITION, chemical_element=CHEMICAL_ELEMENT)

defect_builder_parameters = PointDefectBuilderParameters(center_defect=False)

### 2.2. Create the defects

In [6]:
from mat3ra.made.tools.build.defect import create_defect

material_with_defect_at_site_id = create_defect(builder_parameters=defect_builder_parameters, configuration=defect_configuration_with_site_id)
material_with_defect_at_position = create_defect(builder_parameters=defect_builder_parameters, configuration=defect_configuration_with_position)

## 3. Visualize the Defects

In [7]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect_at_site_id, "title": f"Material with defect at site_id={SITE_ID}"},
           {"material": material_with_defect_at_position, "title": f"Material with defect at position={POSITION}"}],
          rotation="-90x")

GridBox(children=(VBox(children=(Label(value='Ni27 - Original material - rotation: -90x', layout=Layout(align_…

## 4. Pass data to the outside runtime

In [8]:
from utils.jupyterlite import set_data

set_data("materials", [material_with_defect_at_site_id.to_json(), material_with_defect_at_position.to_json()])

Data for materials written to uploads/Cu1 Ni26, Substitution Cu Defect.json
Data for materials written to uploads/Cu1 Ni27, Interstitial Cu Defect.json
